In [1]:
import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [5]:
from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

In [26]:
np.random.seed(2017)

n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

y[12500:] = 1

100%|██████████| 12500/12500 [00:57<00:00, 218.01it/s]


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [28]:
# from keras.applications.resnet50 import ResNet50

# base_model = ResNet50(input_tensor=Input((img_height, img_width, 3)), weights='imagenet', include_top=False)

base_model = InceptionResNetV2(input_tensor=Input((img_height, img_width, 3)), weights='imagenet', include_top=False)

In [29]:
for layers in base_model.layers:
    layers.trainable = False

In [30]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.25)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_valid, y_valid))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 155s 8ms/step - loss: 0.1425 - acc: 0.9495 - val_loss: 8.0236 - val_acc: 0.5022
Epoch 2/5
  400/20000 [..............................] - ETA: 1:52 - loss: 0.0841 - acc: 0.9775

KeyboardInterrupt: 